In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings

documents = SimpleDirectoryReader("data").load_data()
print(documents)
print(documents[0])

[Document(id_='7a3082aa-d6b8-45d7-b1c8-6b4328cf2686', embedding=None, metadata={'file_path': 'd:\\Projects\\nlp-bachelor\\llama3\\data\\info1.txt', 'file_name': 'info1.txt', 'file_type': 'text/plain', 'file_size': 130, 'creation_date': '2024-07-20', 'last_modified_date': '2024-07-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='The monarch butterfly or simply monarch (Danaus plexippus) is a milkweed butterfly (subfamily Danainae) in the family Nymphalidae.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='9bf969e9-0ec4-4659-87e1-f6a7cb94b7e4', embedding=None, metadata={'file_path': 'd:\\Projects\\nlp-bachelor\\llama

In [3]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=200, chunk_overlap=10)
nodes = text_splitter.get_nodes_from_documents(documents=documents)
nodes

[TextNode(id_='e2129e4b-2ae1-42ab-85e0-1940bc75a6d1', embedding=None, metadata={'file_path': 'd:\\Projects\\nlp-bachelor\\llama3\\data\\info1.txt', 'file_name': 'info1.txt', 'file_type': 'text/plain', 'file_size': 130, 'creation_date': '2024-07-20', 'last_modified_date': '2024-07-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7a3082aa-d6b8-45d7-b1c8-6b4328cf2686', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'd:\\Projects\\nlp-bachelor\\llama3\\data\\info1.txt', 'file_name': 'info1.txt', 'file_type': 'text/plain', 'file_size': 130, 'creation_date': '2024-07-20', 'last_modified_date': '2024-07-20'}, hash='7338c181f08cdd85ec9c6238c8278ff678e912657f6b96127ebaee3540828b0d')}, text='The mo

In [19]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [36]:
from llama_index.core import PromptTemplate


# Transform a string into input zephyr-specific input
def completion_to_prompt(completion):
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"


# Transform a list of chat messages into zephyr-specific input
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == "system":
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == "user":
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == "assistant":
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"

    return prompt


import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

Settings.llm = HuggingFaceLLM(
    model_name="OuteAI/Lite-Mistral-150M-v2-Instruct",
    tokenizer_name="OuteAI/Lite-Mistral-150M-v2-Instruct",
    context_window=3900,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    # messages_to_prompt=messages_to_prompt,
    # completion_to_prompt=completion_to_prompt,
    device_map="auto",
    tokenizer_outputs_to_remove=["token_type_ids"]
)

Supplied context_window 3900 is greater than the model's max input size 2048. Disable this warning by setting a lower context_window.


In [37]:
index = VectorStoreIndex.from_documents(documents)

In [38]:
query_engine = index.as_query_engine()
response = query_engine.query("Who wrote the play Romeo and Juliet?")

c:\Users\ADMIN\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\ADMIN\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [39]:
print(response)


The play Romeo and Juliet was written by Antonio Vivaldi.
